## ESG Score 

### Import

In [ ]:
import os
import numpy as np
import random
import pandas as pd
import warnings; warnings.filterwarnings("ignore")

random.seed(1004)
data_path = "../../3. 경서경욱_데이터 및 모델 세이브 파일/dataset/"

### Read Data

In [ ]:
pdde = pd.read_csv(data_path + "origin_dataset/LPOINT_BIG_COMP_02_PDDE.csv")       # 유통사 상품 구매 내역   
demo = pd.read_csv(data_path+"origin_dataset/LPOINT_BIG_COMP_01_DEMO.csv")         # 고객 데모 정보 
br = pd.read_csv(data_path+"origin_dataset/LPOINT_BIG_COMP_05_BR.csv")             # 점포 지역 정보

### Generate Columns

#### $\blacktriangleright$ 거주지 내 구매
EDA를 통해 거주지 내 구매는 온라인 구매로 전환될 때 탄소배출 감소에 도움이 될 것이라 확인하였다. 이에 거주지 내 구매 여부를 나타내는 열을 만들어 ESG Score를 산출한다.

In [ ]:
pdde = pdde.reset_index()
offline = pd.merge(pdde, demo[['cust','zon_hlv']].rename(columns={'zon_hlv':'거주지'}), on='cust')\
          .merge(br[['br_c','zon_hlv']].rename(columns={'zon_hlv':'매장위치'}), on='br_c')
sameplace = offline[offline['거주지']==offline['매장위치']]['index']

pdde['거주지내구매'] = pdde['index'].apply(lambda x: 1 if x in sameplace else 0)
del pdde['index']

#### $\blacktriangleright$ 친환경 상품
롯데지주의 지속가능경영보고서를 바탕으로 저탄소 상품, 중고거래상품, 친환경 포장재 상품 등을 통한 2040년 탄소중립을 목표로 한다. 임의로 해당 상품에 대한 정보를 만들어 ESG Score를 산출한다.

In [ ]:
# 약 10퍼센트 비율로 저탄소상품 라벨링
pdde['저탄소상품'] = [random.choice([0, 0, 0, 0, 0, 0, 0, 0, 0, 1]) for i in range(pdde.shape[0])]

In [ ]:
# 약 10퍼센트 비율로 중고거래상품 라벨링
pdde['중고거래상품'] = [random.choice([0, 0, 0, 0, 0, 0, 0, 0, 0, 1]) for i in range(pdde.shape[0])]

In [ ]:
# 약 20퍼센트 비율로 친환경포장재 사용 상품 라벨링
pdde['친환경포장재'] = [random.choice([0,0,0,0,1]) for i in range(pdde.shape[0])]

#### $\blacktriangleright$ ESG 협력사 지원
2021년에 발표된 K-ESG 평가기준은 S분야의 ESG 협력사 경영, 지원을 거론하며 상생경영을 강조하였다. ESG 협력사의 제품임을 나타내는 임의의 열을 생성하여 ESG Score를 산출한다.

In [ ]:
# 약 20퍼센트 비율로 ESG협력사 제품 라벨링
pdde['ESG협력사제품'] = [random.choice([0,0,0,0,1]) for i in range(pdde.shape[0])]

In [ ]:
pdde.head(5)

### Calculate ESG Score

In [ ]:
ESG = pdde.drop_duplicates(['cust','cop_c','chnl_dv','de_dt','de_hr'])[['cust','cop_c','chnl_dv','de_dt','de_hr']].reset_index(drop=True)
ESG

In [ ]:
ESG = pd.merge(ESG,
               pdde.query('chnl_dv==2').groupby(['cust','cop_c','chnl_dv','de_dt','de_hr'])['buy_ct'].sum().rename('온라인상품수').reset_index(),
               on=['cust','cop_c','chnl_dv','de_dt','de_hr'], how='outer')\
      .merge(pdde.query('거주지내구매==1').groupby(['cust','cop_c','chnl_dv','de_dt','de_hr'])['buy_ct'].sum().rename('거주지내상품수').reset_index(),
               on=['cust','cop_c','chnl_dv','de_dt','de_hr'], how='outer')\
      .merge(pdde.query('저탄소상품==1').groupby(['cust','cop_c','chnl_dv','de_dt','de_hr'])['buy_am'].sum().rename('저탄소상품액').reset_index(),
               on=['cust','cop_c','chnl_dv','de_dt','de_hr'], how='outer')\
      .merge(pdde.query('중고거래상품==1').groupby(['cust','cop_c','chnl_dv','de_dt','de_hr'])['buy_am'].sum().rename('중고거래상품액').reset_index(),
             on=['cust','cop_c','chnl_dv','de_dt','de_hr'], how='outer')\
      .merge(pdde.query('친환경포장재==1').groupby(['cust','cop_c','chnl_dv','de_dt','de_hr'])['buy_am'].sum().rename('친환경포장재상품액').reset_index(),
             on=['cust','cop_c','chnl_dv','de_dt','de_hr'], how='outer')\
      .merge(pdde.query('ESG협력사제품==1').groupby(['cust','cop_c','chnl_dv','de_dt','de_hr'])['buy_am'].sum().rename('ESG협력사상품액').reset_index(),
             on=['cust','cop_c','chnl_dv','de_dt','de_hr'], how='outer')
ESG.fillna(0, inplace=True)
ESG

In [ ]:
ESG_SCORE = pd.concat([ESG[['cust','cop_c','chnl_dv','de_dt','de_hr']],
                       (ESG['온라인상품수'] * 700).rename('온라인상품수_마일리지'),
                       (ESG['거주지내상품수'] * -700).rename('거주지내구매_감점'),
                       (ESG[[i for i in ESG.columns if '액' in i]].sum(axis=1) * 0.01).rename('상품액_마일리지')], axis=1)

ESG_SCORE = pd.merge(ESG_SCORE,
                     ESG_SCORE.set_index(['cust','cop_c','chnl_dv','de_dt','de_hr']).sum(axis=1).rename('ESG_score').reset_index(),
                     on=['cust','cop_c','chnl_dv','de_dt','de_hr'], how='outer')
ESG_SCORE

### save data

In [ ]:
ESG_SCORE.to_csv(data_path + "custom_dataset/ESG_SCORE.csv")